In [ ]:
import oat_python as oat
import plotly.graph_objects as go
import numpy as np

In [ ]:
#   SET PARAMETER VALUES
npoints_per_circle  =   80
maxdis              =   None
maxdim              =   1

#   GENERATE CLOUD FROM RANDOM SEED
cloud               =   []
for seed in range(3):
    circle          =   oat.point_cloud.annulus(npoints=npoints_per_circle, rad0=1, rad1=2.5, random_seed=seed)
    circle[:,0]     +=  2 * np.cos( seed * 2 * np.pi / 4)
    circle[:,1]     +=  2 * np.sin( seed * 2 * np.pi / 4)    
    cloud.append( circle )
cloud               =   np.concatenate(cloud)

#   PLOT
trace               =   go.Scatter(x=cloud[:,0],y=cloud[:,1], mode="markers")
fig                 =   go.Figure([trace])
fig.update_layout( title=dict(text="Three circles, with noise"), width=800, height=800 )
fig.show()

# Compute persistent homology

We compute persistent homology by factoring the boundary matrix.  The following cell generates a sparse distance matrix and feeds it to the persistent homology solver.  The result is a factored boundary matrix.  We will extract information from this matrix in the following cells.

In [ ]:
# the minimum enclosing radius; all homology vanishes above this filtration parameter
enclosing               =   oat.dissimilarity.enclosing_from_cloud(cloud)   

# distance matrix with values over enclosing + 0.0000000001 removed; adding 0.0000000001 avoids problems due to numerical error
dissimilairty_matrix    =   oat.dissimilarity.matrix_from_cloud(            
                                cloud                       =   cloud,
                                dissimilarity_max           =   enclosing + 0.0000000001,
                            )

# build and factor the boundary matrix
factored                =   oat.rust.FactoredBoundaryMatrixVr( 
                                dissimilarity_matrix        =   dissimilairty_matrix,
                                homology_dimension_max      =   1,
                            )

Full details on `FactoredBoundaryMatrixVr` can be retreived with Python's `help` function.

In [ ]:
help(oat.rust.FactoredBoundaryMatrixVr)

In [ ]:
# the minimum enclosing radius; all homology vanishes above this filtration parameter
enclosing               =   oat.dissimilarity.enclosing_from_cloud(cloud)   

# distance matrix with values over enclosing + 0.0000000001 removed; adding 0.0000000001 avoids problems due to numerical error
dissimilairty_matrix    =   oat.dissimilarity.matrix_from_cloud(            
                                cloud               =   cloud,
                                dissimilarity_max   =   enclosing + 0.0000000001,
                            )

# build and factor the boundary matrix
factored                =   oat.rust.FactoredBoundaryMatrixVr( 
                                dissimilarity_matrix        =   dissimilairty_matrix,
                                homology_dimension_max      =   1,
                            )

# Plot the barcode

In [ ]:
homology    =   factored.homology(
                    return_cycle_representatives    =   True,
                    return_bounding_chains          =   True,
                )
fig = oat.plot.pd( homology )
fig.show()

For convenience, export information about the barcode to a data frame.

# Inspect a cycle representatives and bounding chains

The `homology` object is a Pandas data frame containing information about cycles.

In [ ]:
display(homology)

In [ ]:
homology["cycle representative"][280]

In [ ]:
homology["bounding chain"][280]

# Optimize a cycle

Let's optimize the cycle that has birth simplex [9,19]

In [ ]:
# optimize the cycle
optimal     =   factored.optimize_cycle(
                    birth_simplex                   =   homology["birth simplex"][240], 
                    problem_type                    =   "preserve PH basis",
                )

# display the data frame that contains the solution
display(optimal)

# display the data frame for just the optimal cycle (which is contained in the larger data frame)
display(optimal["chain"]["optimal cycle"])

# Plot an optimal cycle

<span style="color:red">Try toggling the legend entries to make different parts of the calculation appear / disappear!</span>.

In [ ]:
optimal             =   factored.optimize_cycle( homology["birth simplex"][240], problem_type="preserve PH basis")

edges_initial = optimal["chain"]["initial cycle"]["simplex"].tolist();
edges_optimal = optimal["chain"]["optimal cycle"]["simplex"].tolist();
triangles = optimal["chain"]["difference in bounding chains"]["simplex"].tolist()
coo = cloud # coo stands for coordinate oracle

traces_initial      = [ oat.plot.edge__trace2d( edge, coo  ) for edge in edges_initial ];
traces_optimal      = [ oat.plot.edge__trace2d( edge, coo  ) for edge in edges_optimal ];
traces_differe      = [ oat.plot.triangle__trace2d( triangle=tri, coo=coo ) for tri in triangles ];
trace_cloud         = go.Scatter(x=cloud[:,0], y=cloud[:,1], mode="markers") # the point cloud

trace_cloud.update(showlegend=True, opacity=0.5, name="Point cloud")
for n, trace in enumerate(traces_initial): trace.update(showlegend=(n==0), legendgroup="init", opacity=0.5, name="Initial cycle", line=dict(color="white",dash="dash"))
for n, trace in enumerate(traces_optimal): trace.update(showlegend=(n==0), legendgroup="opti", opacity=0.5, name="Optimal cycle", line=dict(color="white"))
for n, trace in enumerate(traces_differe): trace.update(showlegend=(n==0), legendgroup="diff", opacity=0.5, name="Bounding difference", hovertext="Simplex "+str(triangles[n]), fillcolor="white" )

fig = go.Figure(data= [trace_cloud] + traces_initial + traces_optimal + traces_differe )
fig.update_layout(title=dict(text="Optimization of a cycle representative"))
# fig.update_layout(height=800,width=850) 

fig.update_layout(
        title=dict(text="Cycle representative and bounding chain"),
        template="plotly_dark",
        height=1200,
        width=1600,
    )

fig.show()

In [ ]:
import pandas as pd

# Assuming you have a dataframe named 'df' with columns A, B, and C

# Filter the dataframe based on the condition A[i] == 1
# filtered_df = np.array(list( homology[ homology['dimension'] == 0 ][['birth', 'death']].values ))

# Extract the birth/death pairs
birth_death_pairs = np.array(list( homology[ homology['dimension'] == 0 ][['birth', 'death']].values ))
print(birth_death_pairs)